# PD score alternatives

Promiscuity Degree(PD) simply counts the Activity cliffs, it can't distinct "true promiscuity" from "apparent promiscuity" such as Target family dependency.

## 1. Count Target family with ChEMBL 

In [106]:
!wget -c https://zenodo.org/record/200393/files/784_Promiscuity_Cliffs.txt

--2017-11-25 10:56:03--  https://zenodo.org/record/200393/files/784_Promiscuity_Cliffs.txt
Resolving zenodo.org... 188.184.65.20
Connecting to zenodo.org|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



In [107]:
import pandas as pd
from IPython.display import HTML, display
from pychembldb import *
import numpy as np
import subprocess
from sklearn.cluster import AffinityPropagation
from sklearn.manifold import MDS
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [170]:
df = pd.read_table("784_Promiscuity_Cliffs.txt")

In [171]:
for idx, row in df.iterrows():
    if idx > 2:
        break
    else:
        print(row["PromiscuityCliff_ID"])
    chemblids = [cid.strip() for cid in row["CPD_1_ChEMBL_TARGETS"].split(",")]
    for chemblid in chemblids:
        r = chembldb.query(Target).filter_by(chembl_id=chemblid).first()
        print(r.chembl_id, r.components[0].classes[0].pref_name)

1
CHEMBL3028 Enzyme
CHEMBL5896 Plant homeodomain
CHEMBL5697 Enzyme
CHEMBL6175 Plant homeodomain
CHEMBL1938210 Plant homeodomain
CHEMBL1938211 Jumonji domain-containing
CHEMBL1938212 Jumonji domain-containing
CHEMBL2163176 Plant homeodomain
CHEMBL2331065 Unclassified protein
CHEMBL2424504 Jumonji domain-containing
CHEMBL5909 Enzyme
2
CHEMBL3028 Enzyme
CHEMBL5896 Plant homeodomain
CHEMBL5697 Enzyme
CHEMBL6175 Plant homeodomain
CHEMBL1938210 Plant homeodomain
CHEMBL1938211 Jumonji domain-containing
CHEMBL1938212 Jumonji domain-containing
CHEMBL2163176 Plant homeodomain
CHEMBL2331065 Unclassified protein
CHEMBL2424504 Jumonji domain-containing
CHEMBL5909 Enzyme
3
CHEMBL3028 Enzyme
CHEMBL5896 Plant homeodomain
CHEMBL5697 Enzyme
CHEMBL6175 Plant homeodomain
CHEMBL1938210 Plant homeodomain
CHEMBL1938211 Jumonji domain-containing
CHEMBL1938212 Jumonji domain-containing
CHEMBL2163176 Plant homeodomain
CHEMBL2331065 Unclassified protein
CHEMBL2424504 Jumonji domain-containing
CHEMBL5909 Enzyme


This approarch does not work well ;-(

## 2. Considering similarity matrix to PD

Calcurate sequence variablility: Ratio of (nuuber of pair whose simirality < 0.7 / total) or Calulate number of Clusters using MDS+AP

In [172]:
def mat2arr(matfile):
    arr = []
    with open(matfile)  as f:
        f.readline()
        for l in f:
            ls = l[:-1].split()
            arr.append([float(e) for e in ls[1:]])
    return arr

In [173]:
def count_desim(mat):
        pd = len(mat[0])
        total = 0
        fd = 0
        for i in range(1, pd):
            for j in range(0, pd-1):
                if mat[i][j] < 0.7:
                    fd += 1
                total += 1
        target_diversity = float(fd) / total
        return target_diversity

In [174]:
computed_chemblid = ""
for idx, row in df.iterrows():
    if computed_chemblid != row["CPD_1_SMILES"]:
        computed_chemblid = row["CPD_1_SMILES"]
        chemblids = [cid.strip() for cid in row["CPD_1_ChEMBL_TARGETS"].split(",")]
        with open("seqs/{}.fa".format(row["PromiscuityCliff_ID"]), "w") as f:
            for chemblid in chemblids:
                r = chembldb.query(Target).filter_by(chembl_id=chemblid).first()
                f.write(">{}\n{}\n".format(chemblid, r.components[0].sequence))
        input = "seqs/{}.fa".format(row["PromiscuityCliff_ID"])
        output = "seqs/{}-out.fa".format(row["PromiscuityCliff_ID"])
        distmat = "seqs/{}.mat".format(row["PromiscuityCliff_ID"])
        if not os.path.exists(distmat):
            subprocess.call(["clustalo", "-i", input, "-o", output, "-v",  "--force", "--distmat-out={}".format(distmat), "--full"])

        mat = mat2arr(distmat)
        mds = MDS(2, max_iter=100, n_init=1, dissimilarity='precomputed')
        mds_data = mds.fit_transform(mat)
        af = AffinityPropagation().fit(mds_data)
        df = pd.DataFrame(mds_data, columns=['x', 'y'])
        intlabels = map(int, af.labels_)
        print("Promiscuity ID: ", idx, "PD: ", len(mat), "TFD: ", max(intlabels))
        #df["label"] = pd.Series(intlabels)
        #plt.figure(idx)
        #sns.lmplot( x="x", y="y", data=df, fit_reg=False, hue='label', legend=False);




Promiscuity ID:  0 PD:  11 TFD:  2
Promiscuity ID:  6 PD:  11 TFD:  2
Promiscuity ID:  42 PD:  11 TFD:  2
Promiscuity ID:  57 PD:  11 TFD:  2
Promiscuity ID:  58 PD:  11 TFD:  2
Promiscuity ID:  62 PD:  11 TFD:  2
Promiscuity ID:  64 PD:  11 TFD:  2
Promiscuity ID:  70 PD:  11 TFD:  2
Promiscuity ID:  91 PD:  11 TFD:  2
Promiscuity ID:  92 PD:  11 TFD:  2
Promiscuity ID:  96 PD:  11 TFD:  2
Promiscuity ID:  97 PD:  11 TFD:  2
Promiscuity ID:  99 PD:  11 TFD:  2
Promiscuity ID:  103 PD:  11 TFD:  2
Promiscuity ID:  157 PD:  11 TFD:  2
Promiscuity ID:  158 PD:  12 TFD:  2
Promiscuity ID:  183 PD:  12 TFD:  2
Promiscuity ID:  194 PD:  12 TFD:  2
Promiscuity ID:  195 PD:  12 TFD:  2
Promiscuity ID:  215 PD:  12 TFD:  2
Promiscuity ID:  265 PD:  12 TFD:  2
Promiscuity ID:  267 PD:  12 TFD:  2
Promiscuity ID:  268 PD:  12 TFD:  2
Promiscuity ID:  270 PD:  12 TFD:  2
Promiscuity ID:  272 PD:  12 TFD:  2
Promiscuity ID:  286 PD:  12 TFD:  2
Promiscuity ID:  289 PD:  12 TFD:  3
Promiscuity ID: 

# Conclusion

Both methods doesn't work well, but the latter approach has a room for improvement. I used global sequence alignment using clustalomega, but I think local alignments (such as BLAST) are suitable method than clustalomega.